Messy code for counting tokens from the experiments.

In [1]:
from collections import Counter

from redel.utils import read_jsonl


# from the docs
def count_tokens(fp):
    tokens_used_prompt = Counter()
    tokens_used_output = Counter()
    for event in read_jsonl(fp):
        if event["type"] == "tokens_used":
            tokens_used_prompt[event["id"]] += event["prompt_tokens"]
            tokens_used_output[event["id"]] += event["completion_tokens"]
    return tokens_used_prompt, tokens_used_output

In [2]:
from pathlib import Path

# define base experiments path
EXPERIMENTS = Path("/Users/andrew/Desktop/Code/_penn/redel-experiments/experiments")

In [3]:
import json


def count_system(fp, root_input_cost_1m, root_output_cost_1m, del_input_cost_1m=0, del_output_cost_1m=0):
    if not fp.exists():
        return 0, 0
    # get all event paths in system
    event_paths = []
    for result in read_jsonl(fp / "results.jsonl"):
        event_paths.append(fp / Path(result["log_dir"]).stem / "events.jsonl")

    n = len(event_paths)
    total_prompt_tokens_root = 0
    total_output_tokens_root = 0
    total_prompt_tokens_del = 0
    total_output_tokens_del = 0
    # for each run,
    for ep in event_paths:
        # find the root node ID
        with open(ep.parent / "state.json") as f:
            state = json.load(f)
            root_id = state["state"][0]["id"]

        # count tokens by root and by del
        prompt_tokens, output_tokens = count_tokens(ep)
        prompt_tokens_root = prompt_tokens[root_id]
        output_tokens_root = output_tokens[root_id]

        total_prompt_tokens_root += prompt_tokens_root
        total_output_tokens_root += output_tokens_root
        total_prompt_tokens_del += prompt_tokens.total() - prompt_tokens_root
        total_output_tokens_del += output_tokens.total() - output_tokens_root

    # m a t h
    # (I should have used a dict but I did this in like 10 minutes don't judge me)
    avg_prompt_tokens_root = total_prompt_tokens_root / n
    avg_output_tokens_root = total_output_tokens_root / n
    avg_prompt_tokens_del = total_prompt_tokens_del / n
    avg_output_tokens_del = total_output_tokens_del / n

    avg_prompt_cost_root = avg_prompt_tokens_root * root_input_cost_1m / 1000000
    avg_output_cost_root = avg_output_tokens_root * root_output_cost_1m / 1000000
    avg_prompt_cost_del = avg_prompt_tokens_del * del_input_cost_1m / 1000000
    avg_output_cost_del = avg_output_tokens_del * del_output_cost_1m / 1000000

    total_cost = (avg_prompt_cost_root + avg_output_cost_root + avg_prompt_cost_del + avg_output_cost_del) * n

    print(f"========== {fp} ==========")
    print(f"Avg prompt tokens root: {avg_prompt_tokens_root} (${avg_prompt_cost_root})")
    print(f"Avg output tokens root: {avg_output_tokens_root} (${avg_output_cost_root})")
    print(f"Avg prompt tokens del: {avg_prompt_tokens_del} (${avg_prompt_cost_del})")
    print(f"Avg output tokens del: {avg_output_tokens_del} (${avg_output_cost_del})")
    print(f"N: {n}")
    print(f"Total cost: ${total_cost}")

    return total_cost, n

In [4]:
count_system(
    EXPERIMENTS / Path("fanoutqa/dev/trial2/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)
count_system(
    EXPERIMENTS / Path("travelplanner/validation/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)
count_system(
    EXPERIMENTS / Path("webarena/test/small-leaf"),
    root_input_cost_1m=5,
    root_output_cost_1m=15,
    del_input_cost_1m=0.5,
    del_output_cost_1m=1.5,
)

========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/dev/trial2/small-leaf ==========
Avg prompt tokens root: 1913.8733766233765 ($0.009569366883116883)
Avg output tokens root: 363.4642857142857 ($0.0054519642857142865)
Avg prompt tokens del: 28171.0 ($0.0140855)
Avg output tokens del: 509.6818181818182 ($0.0007645227272727273)
N: 308
Total cost: $9.200377000000001
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/travelplanner/validation/small-leaf ==========
Avg prompt tokens root: 7912.888888888889 ($0.039564444444444444)
Avg output tokens root: 1018.7333333333333 ($0.015281)
Avg prompt tokens del: 9192.044444444444 ($0.0045960222222222215)
Avg output tokens del: 1121.0833333333333 ($0.001681625)
N: 180
Total cost: $11.0021565
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/webarena/test/small-leaf ==========
Avg prompt tokens root: 55456.268 ($0.27728134)
Avg output tokens root: 171.884 ($0.002578259999

(98.039051, 250)

In [5]:
count_system(EXPERIMENTS / Path("fanoutqa/dev/trial2/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)
count_system(EXPERIMENTS / Path("travelplanner/validation/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)
count_system(EXPERIMENTS / Path("webarena/test/baseline"), root_input_cost_1m=5, root_output_cost_1m=15)

========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/dev/trial2/baseline ==========
Avg prompt tokens root: 65899.60586319218 ($0.3294980293159609)
Avg output tokens root: 324.8371335504886 ($0.004872557003257329)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 307
Total cost: $102.65177
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/travelplanner/validation/baseline ==========
Avg prompt tokens root: 42591.40555555555 ($0.21295702777777775)
Avg output tokens root: 1224.5611111111111 ($0.01836841666666667)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 180
Total cost: $41.63858
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/webarena/test/baseline ==========
Avg prompt tokens root: 145537.47430830039 ($0.727687371541502)
Avg output tokens root: 168.6798418972332 ($0.002530197628458498)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 25

(184.745045, 253)

In [6]:
total_cost_small_leaf = 9.200377 + 11.0021565 + 98.039051
total_cost_baseline = 102.65177 + 41.63858 + 184.745045
print(total_cost_small_leaf, total_cost_baseline)

118.2415845 329.035395


In [7]:
329.035395 / 118.2415845

2.782738377461442

# OpenAI counting

In [8]:
costs_by_system = {
    "full": (5, 15, 5, 15),
    "root-fc": (5, 15, 5, 15),
    "baseline": (5, 15, 0, 0),
    "small-leaf": (5, 15, 0.5, 1.5),
    "small-all": (0.5, 1.5, 0.5, 1.5),
    "small-baseline": (0.5, 1.5, 0, 0),
    "short-context": (5, 15, 5, 15),
    "short-baseline": (5, 15, 0, 0),
}

for system in [
    "full",
    "root-fc",
    "baseline",
    "small-leaf",
    "small-all",
    "small-baseline",
    "short-context",
    "short-baseline",
]:
    benchmark_cost = 0
    for benchmark in ["fanoutqa/dev/trial2", "travelplanner/validation", "webarena/test"]:
        root_in_cost, root_out_cost, del_in_cost, del_out_cost = costs_by_system[system]
        total_cost, _ = count_system(
            EXPERIMENTS / benchmark / system,
            root_input_cost_1m=root_in_cost,
            root_output_cost_1m=root_out_cost,
            del_input_cost_1m=del_in_cost,
            del_output_cost_1m=del_out_cost,
        )
        benchmark_cost += total_cost
    print(f"TOTAL SYSTEM COST: ${benchmark_cost}")

========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/dev/trial2/full ==========
Avg prompt tokens root: 1040.8770226537217 ($0.005204385113268609)
Avg output tokens root: 329.5210355987055 ($0.0049428155339805825)
Avg prompt tokens del: 139570.63754045308 ($0.6978531877022655)
Avg output tokens del: 733.1359223300971 ($0.010997038834951456)
N: 309
Total cost: $222.17020500000004
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/travelplanner/validation/full ==========
Avg prompt tokens root: 10247.433333333332 ($0.05123716666666667)
Avg output tokens root: 999.2222222222222 ($0.014988333333333333)
Avg prompt tokens del: 7518.516666666666 ($0.037592583333333325)
Avg output tokens del: 1628.2611111111112 ($0.024423916666666667)
N: 180
Total cost: $23.083559999999995
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/webarena/test/full ==========
Avg prompt tokens root: 44021.18656716418 ($0.2201059328358209)
Avg

# Claude counting

In [10]:
costs_by_system = {
    "full": (3, 15, 3, 15),
    "root-fc": (3, 15, 3, 15),
    "baseline": (3, 15, 0, 0),
    "small-leaf": (3, 15, 1, 5),
    "small-all": (1, 5, 1, 5),
    "small-baseline": (1, 5, 0, 0),
    "short-context": (3, 15, 3, 15),
    "short-baseline": (3, 15, 0, 0),
}

for system in [
    # "full",
    # "root-fc",
    "baseline",
    # "small-leaf",
    # "small-all",
    # "small-baseline",
    "short-context",
    "short-baseline",
]:
    benchmark_cost = 0
    for benchmark in ["fanoutqa/claude", "travelplanner/claude", "webarena/claude"]:
        root_in_cost, root_out_cost, del_in_cost, del_out_cost = costs_by_system[system]
        total_cost, _ = count_system(
            EXPERIMENTS / benchmark / system,
            root_input_cost_1m=root_in_cost,
            root_output_cost_1m=root_out_cost,
            del_input_cost_1m=del_in_cost,
            del_output_cost_1m=del_out_cost,
        )
        benchmark_cost += total_cost
    print(f"TOTAL SYSTEM COST: ${benchmark_cost}")

========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/claude/baseline ==========
Avg prompt tokens root: 207894.3074433657 ($0.623682922330097)
Avg output tokens root: 581.2556634304207 ($0.008718834951456311)
Avg prompt tokens del: 0.0 ($0.0)
Avg output tokens del: 0.0 ($0.0)
N: 309
Total cost: $195.412143
TOTAL SYSTEM COST: $195.412143
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/claude/short-context ==========
Avg prompt tokens root: 5315.0 ($0.015945)
Avg output tokens root: 495.7605177993527 ($0.007436407766990291)
Avg prompt tokens del: 51123.95469255663 ($0.1533718640776699)
Avg output tokens del: 1808.7216828478965 ($0.02713082524271845)
N: 309
Total cost: $63.000186
TOTAL SYSTEM COST: $63.000186
========== /Users/andrew/Desktop/Code/_penn/redel-experiments/experiments/fanoutqa/claude/short-baseline ==========
Avg prompt tokens root: 19453.951612903227 ($0.05836185483870968)
Avg output tokens root: 600.129032258